In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu117.html
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu117.html
^C
Traceback (most recent call last):
  File "/home/hamid/python3-virtualenv/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/home/hamid/python3-virtualenv/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/home/hamid/python3-virtualenv/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/home/hamid/python3-virtualenv/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/home/hamid/python3-virtualenv/lib/python3.11/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_

In [ ]:
# If you have installed dgl-cuXX package, please uninstall it first.
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
^C
ERROR: Operation cancelled by user
Looking in links: https://data.dgl.ai/wheels-test/repo.html


In [ ]:
!pip install rdkit-pypi

In [ ]:
!pip install dgllife==0.2.8
!pip install scikit-learn
!pip install yacs
!pip install prettytable



In [ ]:
from models import DrugBAN
from time import time
from utils import set_seed, graph_collate_func, mkdir,graph_collate_func2
from configs import get_cfg_defaults
from dataloader import DTIDataset, MultiDataLoader, DTIDataset2
from torch.utils.data import DataLoader
from trainer import Trainer
from domain_adaptator import Discriminator
import torch
import argparse
import warnings, os
import torch
import torch.nn as nn
import copy
import os
import numpy as np
from tqdm import tqdm
from rdkit.Chem import AllChem
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
from dgllife.model.gnn import GCN
import pandas as pd
from torch.nn.utils.weight_norm import weight_norm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg_path = "DrugBAN_DA.yaml"
comet_support = False
cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
torch.cuda.empty_cache()
warnings.filterwarnings("ignore")
set_seed(cfg.SOLVER.SEED)
mkdir(cfg.RESULT.OUTPUT_DIR)
experiment = None
print(f"Config yaml: {cfg_path}")
print(f"Running on: {device}")
print(f"Hyperparameters:")
dict(cfg)

Config yaml: DrugBAN_DA.yaml
Running on: cuda
Hyperparameters:


{'DRUG': CfgNode({'NODE_IN_FEATS': 75, 'PADDING': True, 'HIDDEN_LAYERS': [128, 128, 128], 'NODE_IN_EMBEDDING': 128, 'MAX_NODES': 290}),
 'PROTEIN': CfgNode({'NUM_FILTERS': [128, 128, 128], 'KERNEL_SIZE': [3, 6, 9], 'EMBEDDING_DIM': 128, 'PADDING': True}),
 'BCN': CfgNode({'HEADS': 2}),
 'DECODER': CfgNode({'NAME': 'MLP', 'IN_DIM': 256, 'HIDDEN_DIM': 512, 'OUT_DIM': 128, 'BINARY': 2}),
 'SOLVER': CfgNode({'MAX_EPOCH': 100, 'BATCH_SIZE': 8, 'NUM_WORKERS': 0, 'LR': 0.0001, 'DA_LR': 5e-05, 'SEED': 666}),
 'RESULT': CfgNode({'OUTPUT_DIR': './result', 'SAVE_MODEL': True}),
 'DA': CfgNode({'TASK': True, 'METHOD': 'CDAN', 'USE': True, 'INIT_EPOCH': 8, 'LAMB_DA': 1, 'RANDOM_LAYER': True, 'ORIGINAL_RANDOM': True, 'RANDOM_DIM': 256, 'USE_ENTROPY': False}),
 'COMET': CfgNode({'WORKSPACE': 'pz-white', 'PROJECT_NAME': 'DrugBAN_CDAN', 'USE': False, 'TAG': 'DA'})}

In [ ]:
df_train = pd.read_csv("source_train.csv")
df_val = pd.read_csv("target_train.csv")
df_test = pd.read_csv("target_test.csv")

In [ ]:
df_train.shape, df_val.shape, df_test.shape

((9766, 5), (88, 3), (88, 3))

In [ ]:
from rdkit.Chem import AllChem
import sys
import numpy as np
sys.path.append('/usr/local/lib/python3.7/site-packages/')

try:
  from rdkit import Chem
  from rdkit.Chem.Draw import IPythonConsole
except ImportError:
  print('Stopping RUNTIME. Colaboratory will restart automatically. Please run again.')
  exit()
df_list = [df_train,df_val, df_test]
for dfs in df_list:

    x_batch11 = []
    # y = torch.Tensor([y])
    smiles2 = dfs.iloc[dfs.index]['SMILES']
    batch_smiles2 = list(smiles2)
    for item in batch_smiles2:

        m1 = Chem.MolFromSmiles(str(item))
        fp1 = AllChem.GetMorganFingerprintAsBitVect(m1, radius=2, nBits=1024)
        x = np.array(fp1, dtype=np.float64)
        x_batch11.append(x)
    dfs['fcfp'] = x_batch11

In [ ]:
#train_emb = pd.read_parquet("teacher_biosnap_full_graph_juststructure.parquet")

In [ ]:
#train_emb = pd.read_csv("teacher_bindingDB_ourclusteringV6_juststructure_emb_256_source_train.csv")
train_emb = pd.read_csv("teacher_biosnap_cluster_juststructure_emb_256_source_train.csv")


In [ ]:
train_emb['Array'] = train_emb.apply(lambda row: np.array(row), axis=1)

# Drop all columns except the 'Array' column
train_emb.drop(train_emb.columns.difference(['Array']), axis=1, inplace=True)

df_train['teacher_emb'] = train_emb['Array']

In [ ]:
#dataFolder = data


train_dataset = DTIDataset2(df_train.index.values, df_train)
val_dataset = DTIDataset(df_val.index.values, df_val)
test_dataset = DTIDataset(df_test.index.values, df_test)

params1 = {'batch_size': cfg.SOLVER.BATCH_SIZE, 'shuffle': True, 'num_workers': cfg.SOLVER.NUM_WORKERS, 'drop_last': True, 'collate_fn': graph_collate_func}
params2 = {'batch_size': cfg.SOLVER.BATCH_SIZE, 'shuffle': True, 'num_workers': cfg.SOLVER.NUM_WORKERS, 'drop_last': True, 'collate_fn': graph_collate_func2}
source_generator = DataLoader(train_dataset, **params2)
target_generator = DataLoader(val_dataset, **params1)
n_batches = max(len(source_generator), len(target_generator))
multi_generator = MultiDataLoader(dataloaders=[source_generator, target_generator], n_batches=n_batches)
training_generator = DataLoader(train_dataset, **params2)
params1['shuffle'] = False
params1['drop_last'] = False
val_generator = DataLoader(val_dataset,**params1)
test_generator = DataLoader(test_dataset,**params1)

In [ ]:
model = DrugBAN(**cfg).to(device)
opt = torch.optim.Adam(model.parameters(), lr=cfg.SOLVER.LR)
if torch.cuda.is_available():
  torch.backends.cudnn.benchmark = True

In [ ]:
if cfg.DA.USE:
        if cfg["DA"]["RANDOM_LAYER"]:
            domain_dmm = Discriminator(input_size=cfg["DA"]["RANDOM_DIM"], n_class=cfg["DECODER"]["BINARY"]).to(device)
        else:
            domain_dmm = Discriminator(input_size=cfg["DECODER"]["IN_DIM"] * cfg["DECODER"]["BINARY"],
                                       n_class=cfg["DECODER"]["BINARY"]).to(device)
        # params = list(model.parameters()) + list(domain_dmm.parameters())
        opt = torch.optim.Adam(model.parameters(), lr=cfg.SOLVER.LR)
        opt_da = torch.optim.Adam(domain_dmm.parameters(), lr=cfg.SOLVER.DA_LR)
else:
        opt = torch.optim.Adam(model.parameters(), lr=cfg.SOLVER.LR)
torch.backends.cudnn.benchmark = True

In [ ]:
#Test inductive on BioSNAP
trainer = Trainer(model, opt, device, multi_generator, val_generator, test_generator, opt_da=opt_da,
                          discriminator=domain_dmm,
                          experiment=None, **cfg)
result = trainer.train()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 305/305 [01:05<00:00,  4.64it/s]


Training at Epoch 1 with model training loss 1.9994855681403738
Validation at Epoch 1 with validation loss 0.7267333805038217  AUROC 0.6143424972438967 AUPRC 0.5805846379724615


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 305/305 [01:04<00:00,  4.75it/s]


Training at Epoch 2 with model training loss 1.8741347269933732
Validation at Epoch 2 with validation loss 0.8288647229211372  AUROC 0.5870149007392467 AUPRC 0.5810576235837478


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 305/305 [01:04<00:00,  4.74it/s]


...

Training at Epoch 38 model training loss 1.4560308604943948, da loss 1.3808568266571546, total training loss 2.8368876816796473, DA lambda 1
Validation at Epoch 38 with validation loss 1.421268832265285  AUROC 0.6749820895077798 AUPRC 0.7206964845294539


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 305/305 [01:20<00:00,  3.81it/s]


Training at Epoch 39 model training loss 1.4513702756068745, da loss 1.3832354694116311, total training loss 2.834605743064255, DA lambda 1
Validation at Epoch 39 with validation loss 1.4497220686130357  AUROC 0.6763349889710274 AUPRC 0.7205192406939942


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 305/305 [01:20<00:00,  3.81it/s]


Training at Epoch 40 model training loss 1.4558530268121939, da loss 1.381860137767479, total training loss 2.8377131626254224, DA lambda 1
Validation at Epoch 40 with validation loss 1.4375048052323491  AUROC 0.6702018042104323 AUPRC 0.7156365078569272
Test at Best Model of Epoch 28 with test loss 1.0727061012695576  AUROC 0.7080472624720412 AUPRC 0.7508040936442822 f1-score 0.6770292747698895 Specificity 0.7472527472527473 Accuracy 0.6438809261300992 Thred_optim 0.06892354786396027
